In [ ]:
import os
import json
import keras
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
import cv2

In [ ]:
# train_dir = '../input/ranzcr-clip-catheter-line-classification/train/'

# df = pd.read_csv('../input/ranzcr-clip-catheter-line-classification/train.csv')

# classes = [col for col in df.columns if col not in ['StudyInstanceUID','PatientID']]
# n_class = len(classes)
# print(classes)
# print(n_class)

# df['name'] = df['StudyInstanceUID'].apply(lambda x:x+'.jpg')

In [ ]:
# from sklearn.model_selection import train_test_split

# train, val = train_test_split(df, test_size = 0.1, random_state = 42, shuffle=True)

# from keras.preprocessing.image import ImageDataGenerator

# BS = 16
# IMG_SIZE = 260
# SIZE = (IMG_SIZE,IMG_SIZE)

# datagen = ImageDataGenerator(
#             preprocessing_function = tf.keras.applications.efficientnet.preprocess_input,
#             rescale=1/255.0)

# train_set=datagen.flow_from_dataframe(dataframe=train,
#                                             directory=train_dir,
#                                             x_col="name",
#                                             y_col=classes,
#                                             batch_size=BS,
#                                             seed=42,
#                                             shuffle=True,
#                                             class_mode="raw",
#                                             target_size=SIZE)

# val_set=datagen.flow_from_dataframe(dataframe=val,
#                                             directory=train_dir,
#                                             x_col="name",
#                                             y_col=classes,
#                                             batch_size=BS,
#                                             seed=42,
#                                             shuffle=True,
#                                             class_mode="raw",
#                                             target_size=SIZE)

In [ ]:
# from tensorflow.keras.optimizers import RMSprop, Adam, SGD
# from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
# from keras.models import Sequential
# from tensorflow.keras.layers import GlobalAveragePooling2D, Flatten, Dense, Dropout, BatchNormalization

    
# base = tf.keras.applications.EfficientNetB2(weights='imagenet',
#                                             include_top=False, 
#                                             input_shape=(IMG_SIZE, IMG_SIZE, 3),
#                                             drop_connect_rate=0.5)

# model = Sequential()

# model.add(base)

# model.add(GlobalAveragePooling2D())
# model.add(Dense(n_class, activation = 'sigmoid'))

# model.compile(loss='binary_crossentropy', optimizer='adam', 
#               metrics= [tf.keras.metrics.AUC(multi_label=True)])
# model.summary()

In [ ]:
model = tf.keras.models.load_model('../input/trainedmodel3/bestmodel (15).h5')
model.summary()

In [ ]:
# from tensorflow.keras.optimizers import RMSprop, Adam, SGD
# from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau

# checkpoint_cb = ModelCheckpoint("bestmodel.h5",
#                                 save_best_only=True,
#                                 monitor='val_auc',
#                                 mode='max',
#                                 verbose=1)

# reduce_lr = ReduceLROnPlateau(monitor='val_auc',
#                               factor=0.2,
#                               patience=3,
#                               min_lr=1e-6,
#                               mode='max',
#                               verbose=1)

In [ ]:
# STEP_SIZE_TRAIN = train_set.n//train_set.batch_size
# STEP_SIZE_VALID = val_set.n//val_set.batch_size

# EPOCHS = 10

# history = model.fit(train_set,
#                     validation_data=val_set,
#                     epochs=EPOCHS,
#                     batch_size=BS,
#                     steps_per_epoch=STEP_SIZE_TRAIN,
#                     validation_steps=STEP_SIZE_VALID,
#                     callbacks=[checkpoint_cb, reduce_lr])

# model.save('model.h5')

In [ ]:
sub = pd.read_csv('../input/ranzcr-clip-catheter-line-classification/sample_submission.csv')
test_dir = '../input/ranzcr-clip-catheter-line-classification/test/'

BS = 16
IMG_SIZE = 260
SIZE = (IMG_SIZE,IMG_SIZE)

classes = ['ETT - Abnormal',
         'ETT - Borderline',
         'ETT - Normal',
         'NGT - Abnormal',
         'NGT - Borderline',
         'NGT - Incompletely Imaged',
         'NGT - Normal',
         'CVC - Abnormal',
         'CVC - Borderline',
         'CVC - Normal',
         'Swan Ganz Catheter Present']

results = []

for img_name in sub['StudyInstanceUID']:
    img = cv2.imread(test_dir + img_name + '.jpg')
    img = cv2.resize(img, (IMG_SIZE, IMG_SIZE))
    img = np.array(img)
    img = img / 255.0
    img = img.reshape(1, IMG_SIZE, IMG_SIZE, 3)
    prediction = model.predict(img)
    prediction = prediction[0]
    results.append(prediction)
    
pred_df = pd.DataFrame(columns=classes,data=results, index=sub.index)
pred_df = pd.concat([sub['StudyInstanceUID'],pred_df],axis=1)
pred_df.to_csv('submission.csv',index=False)